In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc
from keras.models import Sequential
from keras.layers import Dense,Activation,BatchNormalization,Dropout
from keras.callbacks import ModelCheckpoint,EarlyStopping,Callback
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import SGD
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.metrics import roc_curve,auc,log_loss, roc_auc_score
from bayes_opt import BayesianOptimization
from sklearn.decomposition import PCA
import lightgbm as lgb

In [71]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [4]:
train.describe()

,id,target,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
count,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,...,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,124.500000,0.640000,0.023292,-0.026872,0.167404,0.001904,0.001588,-0.007304,0.032052,0.078412,...,0.044652,0.126344,0.018436,-0.012092,-0.065720,-0.106112,0.046472,0.006452,0.009372,-0.128952
std,72.312977,0.480963,0.998354,1.009314,1.021709,1.011751,1.035411,0.955700,1.006657,0.939731,...,1.011416,0.972567,0.954229,0.960630,1.057414,1.038389,0.967661,0.998984,1.008099,0.971219
min,0.000000,0.000000,-2.319000,-2.931000,-2.477000,-2.359000,-2.566000,-2.845000,-2.976000,-3.444000,...,-2.804000,-2.443000,-2.757000,-2.466000,-3.287000,-3.072000,-2.634000,-2.776000,-3.211000,-3.500000
25%,62.250000,0.000000,-0.644750,-0.739750,-0.425250,-0.686500,-0.659000,-0.643750,-0.675000,-0.550750,...,-0.617000,-0.510500,-0.535750,-0.657000,-0.818500,-0.821000,-0.605500,-0.751250,-0.550000,-0.754250
50%,124.500000,1.000000,-0.015500,0.057000,0.184000,-0.016500,-0.023000,0.037500,0.060500,0.183500,...,0.067500,0.091000,0.057500,-0.021000,-0.009000,-0.079500,0.009500,0.005500,-0.009000,-0.132500
75%,186.750000,1.000000,0.677000,0.620750,0.805000,0.720000,0.735000,0.660500,0.783250,0.766250,...,0.797250,0.804250,0.631500,0.650250,0.739500,0.493000,0.683000,0.794250,0.654250,0.503250
max,249.000000,1.000000,2.567000,2.419000,3.392000,2.771000,2.901000,2.793000,2.546000,2.846000,...,2.865000,2.801000,2.736000,2.596000,2.226000,3.131000,3.236000,2.626000,3.530000,2.771000


In [ ]:
test.describe().loc[:,[i for i in l]]

In [47]:
l=list(train.corr()['target'].sort_values(ascending=True)[:5].index)

In [48]:
l

['217', '117', '91', '295', '73']

In [49]:
train.describe().loc[:,[i for i in l]]

,217,117,91,295,73
count,250.000000,250.000000,250.000000,250.000000,250.000000
mean,0.041676,0.007536,-0.067724,-0.106112,0.005072
std,1.016528,1.074398,1.001998,1.038389,1.021526
min,-2.678000,-4.270000,-2.909000,-3.072000,-2.868000
25%,-0.661500,-0.733500,-0.660000,-0.821000,-0.717250
50%,0.078000,-0.011500,-0.043500,-0.079500,0.054500
75%,0.693000,0.805750,0.518500,0.493000,0.705250
max,2.743000,3.100000,2.493000,3.131000,2.151000


In [58]:
l2=list(train.corr()['target'].sort_values(ascending=False)[1:6].index)
l2

['33', '65', '24', '183', '199']

In [59]:
test.describe().loc[:,[i for i in l2]]

,33,65,24,183,199
count,19750.000000,19750.000000,19750.000000,19750.000000,19750.000000
mean,-0.002511,-0.007923,0.003181,0.009558,-0.006264
std,0.995098,1.006548,1.005888,1.005222,1.003238
min,-4.311000,-3.732000,-4.032000,-3.954000,-4.004000
25%,-0.675000,-0.686750,-0.680000,-0.671000,-0.684000
50%,-0.005000,-0.002000,0.003000,0.004500,-0.003000
75%,0.671750,0.671000,0.676750,0.681750,0.671000
max,3.966000,4.012000,4.094000,3.913000,4.242000


In [60]:
train.describe().loc[:,[i for i in l2]]

,33,65,24,183,199
count,250.000000,250.000000,250.000000,250.000000,250.000000
mean,-0.134460,0.003796,-0.128252,0.005128,-0.079264
std,1.015563,0.983879,0.997026,0.969452,1.039407
min,-2.913000,-2.799000,-3.107000,-3.118000,-3.521000
25%,-0.829000,-0.636000,-0.779500,-0.710500,-0.853750
50%,-0.216500,0.022000,-0.165500,-0.013500,-0.017000
75%,0.557750,0.687000,0.579250,0.588750,0.674750
max,2.649000,3.163000,2.927000,2.943000,2.185000


In [68]:
pd.Series.corr(train.loc[:,'199'],test.loc[:,'199'])

0.008344638302215652

In [40]:
l2

target    1.000000
33        0.373608
65        0.293846
24        0.173096
183       0.164146
199       0.159442
Name: target, dtype: float64

In [72]:
train['target'].value_counts()

1.0    160
0.0     90
Name: target, dtype: int64

In [73]:
X=train.drop(['target','id'],axis=1,inplace=False)

In [74]:
X_test=test.drop('id',axis=1,inplace=False)

In [75]:
y=train['target']


In [ ]:
X=np.array(X)
y=np.array(y)

In [ ]:
# from sklearn.model_selection import LeaveOneOut
# loo = LeaveOneOut()
# loo.get_n_splits(X)
# for train_index, valid_index in loo.split(X):
#     #print("TRAIN:", train_index, "Valid:", valid_index)
#     X_train, X_valid = X[train_index], X[valid_index]
#     y_train, y_valid = y[train_index], y[valid_index]
    

In [9]:
X_valid.shape

NameError: name 'X_valid' is not defined

In [ ]:
# np.sum(pca.explained_variance_ratio_)

In [19]:
from numba import jit
#@jit
def augment(train,num_n=1,num_p=2):
    newtrain=[train]
    
    n=train[train.target==0]
    for i in range(num_n):
        newtrain.append(n.apply(lambda x:x.values.take(np.random.permutation(len(n)))))
    
    for i in range(num_p):
        p=train[train.target>0]
        newtrain.append(p.apply(lambda x:x.values.take(np.random.permutation(len(p)))))
    return pd.concat(newtrain)

In [ ]:
trn_data

#### RF

In [96]:
param = {
   "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    'n_estimators':100,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 1,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    "bagging_seed" : 10,
    "verbosity" : 1,
}



In [ ]:
params = {'boosting': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary',
          'nthread': -1, 
          'n_estimators':500,
          'num_leaves': 10,
          'learning_rate': 0.1,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'auc'}

gridParams = {
    'learning_rate': [0.01,0.05,0.1,0.01,0.03],
    'n_estimators': [500,800,1000],
    'num_leaves': [5,10,20,30,40,50,60],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65, 0.66,0.5,0.8],
    'subsample' : [0.7,0.75,0.5,0.3],
    'reg_alpha' : [0.1,0.5,0.8,1,1.2,1.5,2],
    'reg_lambda' : [0.1,0.5,0.8,1,1.2,1.4,1.5,2],
    'scale_pos_weight': [0.5,0.8,1],
    'min_child_weight':[2,7,10,20]
    }

mdl = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'binary',
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          n_estimators=params['n_estimators'],
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=5,
                    n_jobs=-1)
# Run the grid
grid.fit(X, y)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)



In [103]:
param={"metric" : "auc",'boosting_type': 'gbdt', 'colsample_bytree': 0.5, 'learning_rate': 0.03, 'n_estimators': 500, 'num_leaves': 5, 'objective': 'binary', 'random_state': 501, 'reg_alpha': 0.1, 'reg_lambda': 1.5, 'subsample': 0.7}

In [105]:
from sklearn.model_selection import RepeatedStratifiedKFold
result=np.zeros(X_test.shape[0])

rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10,random_state=11)
for counter,(train_index, valid_index) in enumerate(rskf.split(X, y),1):
    print (counter)
    
    #Train data
    t=train.iloc[train_index]
    #t=augment(t)
    trn_data = lgb.Dataset(t.drop(["target",'id'],axis=1,inplace=False), label=t.target)
    
    #Validation data
    v=train.iloc[valid_index]
    val_data = lgb.Dataset(v.drop(["target",'id'],axis=1,inplace=False), label=v.target)
    
    #Training
    model = lgb.train(param, trn_data, 1000000, valid_sets = [val_data], verbose_eval=300, early_stopping_rounds = 3000)
    result +=model.predict(X_test)

1
Training until validation scores don't improve for 3000 rounds.


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[300]	valid_0's auc: 0.659722
Did not meet early stopping. Best iteration is:
[23]	valid_0's auc: 0.75
2
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.763889
Did not meet early stopping. Best iteration is:
[44]	valid_0's auc: 0.819444
3
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.694444
Did not meet early stopping. Best iteration is:
[377]	valid_0's auc: 0.715278
4
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.715278
Did not meet early stopping. Best iteration is:
[103]	valid_0's auc: 0.805556
5
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.826389
Did not meet early stopping. Best iteration is:
[15]	valid_0's auc: 0.9375
6
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.75
Did not meet early stopping. Best iteration is:
[61]	valid_0's auc: 0.826389
7
Training until validation scores d

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[300]	valid_0's auc: 0.763889
Did not meet early stopping. Best iteration is:
[429]	valid_0's auc: 0.798611
12
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.576389
Did not meet early stopping. Best iteration is:
[15]	valid_0's auc: 0.722222
13
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.861111
Did not meet early stopping. Best iteration is:
[466]	valid_0's auc: 0.895833
14
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.819444
Did not meet early stopping. Best iteration is:
[32]	valid_0's auc: 0.868056
15
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.708333
Did not meet early stopping. Best iteration is:
[378]	valid_0's auc: 0.736111
16
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.652778
Did not meet early stopping. Best iteration is:
[2]	valid_0's auc: 0.756944
17
Training until val

...

[300]	valid_0's auc: 0.659722
Did not meet early stopping. Best iteration is:
[19]	valid_0's auc: 0.715278
72
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.708333
Did not meet early stopping. Best iteration is:
[389]	valid_0's auc: 0.729167
73
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.75
Did not meet early stopping. Best iteration is:
[13]	valid_0's auc: 0.798611
74
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.541667
Did not meet early stopping. Best iteration is:
[5]	valid_0's auc: 0.645833
75
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.583333
Did not meet early stopping. Best iteration is:
[4]	valid_0's auc: 0.694444
76
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.875
Did not meet early stopping. Best iteration is:
[477]	valid_0's auc: 0.888889
77
Training until validation s

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[300]	valid_0's auc: 0.708333
Did not meet early stopping. Best iteration is:
[366]	valid_0's auc: 0.729167
82
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.743056
Did not meet early stopping. Best iteration is:
[377]	valid_0's auc: 0.770833
83
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.826389
Did not meet early stopping. Best iteration is:
[20]	valid_0's auc: 0.9375
84
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.701389
Did not meet early stopping. Best iteration is:
[43]	valid_0's auc: 0.722222
85
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.631944
Did not meet early stopping. Best iteration is:
[74]	valid_0's auc: 0.722222
86
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.576389
Did not meet early stopping. Best iteration is:
[29]	valid_0's auc: 0.604167
87
Training until valid

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[300]	valid_0's auc: 0.736111
Did not meet early stopping. Best iteration is:
[5]	valid_0's auc: 0.861111
92
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.805556
Did not meet early stopping. Best iteration is:
[21]	valid_0's auc: 0.881944
93
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.666667
Did not meet early stopping. Best iteration is:
[415]	valid_0's auc: 0.701389
94
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.625
Did not meet early stopping. Best iteration is:
[9]	valid_0's auc: 0.694444
95
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.826389
Did not meet early stopping. Best iteration is:
[448]	valid_0's auc: 0.847222
96
Training until validation scores don't improve for 3000 rounds.
[300]	valid_0's auc: 0.763889
Did not meet early stopping. Best iteration is:
[16]	valid_0's auc: 0.8125
97
Training until validation

In [106]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = result/counter
#filename="{:%Y-%m-%d_%H_%M}_sub.csv".format(datetime.now())
submission.to_csv('submission_with_augm.csv', index=False)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Since there was no preprocessing, we didn't need a pipeline here. Used anyway as best practice
modeling_pipeline = make_pipeline(RandomForestClassifier(n_estimators =10000,max_depth=4,max_features ='sqrt',n_jobs=-1))
cv_scores = RepeatedStratifiedKFold(modeling_pipeline, X_tr, y_tr, scoring='roc_auc',cv=20)
for counter,(train_index, valid_index) in enumerate(cv_scores.split(train, train.target),1):
    
    
print("Cross-val auc : %f" %cv_scores.mean())
print("Cross-val std : %f" %cv_scores.std())
print("Cross-val :",cv_scores)